<a href="https://colab.research.google.com/github/bonareri/Bitcoin-Prediction-Analysis/blob/main/data_collection_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1: Install Required Libraries

In [ ]:
!pip install yfinance

## Importing Libraries

In [ ]:
# Data handling and visualization
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime

# For time series analysis and statistical modeling
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# For sentiment analysis
from textblob import TextBlob
import requests
from bs4 import BeautifulSoup

In [ ]:
end = datetime.now()
start = datetime(end.year - 15, end.month, end.day)
stock = 'BTC-USD'
df = yf.download(stock, start=start, end=end)

[*********************100%***********************]  1 of 1 completed


In [ ]:
df.head()

Price,Close,High,Low,Open,Volume
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,
2014-09-17,457.334015,468.174011,452.421997,465.864014,21056800
2014-09-18,424.440002,456.859985,413.104004,456.859985,34483200
2014-09-19,394.795990,427.834991,384.532013,424.102997,37919700
2014-09-20,408.903992,423.295990,389.882996,394.673004,36863600
2014-09-21,398.821014,412.425995,393.181000,408.084991,26580100


## Data Cleaning

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3793 entries, 2014-09-17 to 2025-02-03
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   (Close, BTC-USD)   3793 non-null   float64
 1   (High, BTC-USD)    3793 non-null   float64
 2   (Low, BTC-USD)     3793 non-null   float64
 3   (Open, BTC-USD)    3793 non-null   float64
 4   (Volume, BTC-USD)  3793 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 177.8 KB


Flatten the MultiIndex: The columns have a MultiIndex format (Column Name, Coin Ticker).

In [ ]:
df.columns = df.columns.get_level_values(0)  # This will get only the first part of the tuple

Rename the columns: Renaming the columns to make them more readable. Removing the ticker from the column names and just keeping the price-related terms.



In [ ]:
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

In [ ]:
# Check the cleaned data
print(df.head())

                 Close        High         Low        Open    Volume
Date                                                                
2014-09-17  457.334015  468.174011  452.421997  465.864014  21056800
2014-09-18  424.440002  456.859985  413.104004  456.859985  34483200
2014-09-19  394.795990  427.834991  384.532013  424.102997  37919700
2014-09-20  408.903992  423.295990  389.882996  394.673004  36863600
2014-09-21  398.821014  412.425995  393.181000  408.084991  26580100


## Save Preprocessed Data

At the end of the data preprocessing steps in 1_data_collection_preprocessing.ipynb, save the cleaned and preprocessed data (Bitcoin price data in this case) to a CSV file or pickled format. This will allow you to import the data into the ARIMA and SARIMA models in the next notebook.

In [ ]:
# Save cleaned data to a CSV file after preprocessing
df.to_csv('preprocessed_bitcoin_data.csv')